# **USO DEL MODELO - TEST**

In [ ]:
import json
from tensorflow.keras.models import load_model
from utils import midi_to_sequence, sequence_to_midi
import numpy as np
import random
from music21 import stream, note, chord, converter

**Se carga el modelo**

In [2]:
model = load_model("models/best_model.h5")
with open("models/token2idx.json", 'r') as f:
    token2idx = json.load(f)

VOCAB_SIZE = len(token2idx) + 1

**Funciones auxiliares**

In [3]:

def sequence_to_tokens(seq_indices, idx2token):
    return [idx2token.get(i, "<UNK>") for i in seq_indices]

def build_stream_from_sequence_tokens(seq_tokens):
    """Construye music21.Stream a partir de lista de tokens (C4 o C4.E4.G4)."""
    S = stream.Part()
    S.append(stream.Measure())
    offset = 0
    for t in seq_tokens:
        if "." in t:
            notes = [note.Note(int(n)) if n.isdigit() else note.Note(n) for n in t.split(".")]
            c = chord.Chord(notes)
            c.quarterLength = 1.0
            c.offset = offset
            S.append(c)
        else:
            n = note.Note(t) if not t.isdigit() else note.Note(int(t))
            n.quarterLength = 1.0
            n.offset = offset
            S.append(n)
        offset += 1
    sc = stream.Score()
    sc.insert(0, S)
    return sc

def generate_music_auto(model, seed_seq_indices, gen_len=200, temperature=1.0):
    """Genera índices (no tokens) autoregresivamente usando modelo keras.
       temperature >0 controla aleatoriedad: 1.0 = sin cambio, <1.0 menos variedad, >1.0 más."""
    SEQ_LEN_INPUT_ACTUAL = model.input_shape[1] 
    generated = list(seed_seq_indices)
    
    for _ in range(gen_len):
        input_seq = np.array([generated[-SEQ_LEN_INPUT_ACTUAL:]]) 
        
        preds = model.predict(input_seq, verbose=0)[0]
        
        preds = np.log(preds + 1e-9) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_idx = np.random.choice(range(VOCAB_SIZE), p=preds)
        
        generated.append(next_idx)
    return generated

## **Función para generar música**

Genera índices que luego hay que pasar a tokens y de último a MIDI

In [4]:
def generate_music_auto(model, seed_seq_indices, gen_len=200, temperature=1.0):
    """Genera índices (no tokens) autoregresivamente usando modelo keras.
       temperature >0 controla aleatoriedad: 1.0 = sin cambio, <1.0 menos variedad, >1.0 más."""
    SEQ_LEN_INPUT_ACTUAL = model.input_shape[1] 
    generated = list(seed_seq_indices)
    
    for _ in range(gen_len):
        input_seq = np.array([generated[-SEQ_LEN_INPUT_ACTUAL:]]) 
        
        preds = model.predict(input_seq, verbose=0)[0]
        
        preds = np.log(preds + 1e-9) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_idx = np.random.choice(range(VOCAB_SIZE), p=preds)
        
        generated.append(next_idx)
    return generated

In [5]:
def encode_seed(seed_notes, token2idx):
    seed_encoded = []
    not_founded = 0
    for n in seed_notes:
        if n in token2idx:
            #print(type(n))
            seed_encoded.append(token2idx[n])
        else:
            not_founded += 1
    print(f"Notas no encontradas: {not_founded}")
    return seed_encoded

In [6]:
# Elegir seed:
seed_notes = midi_to_sequence("data/lullaby-piano.mid")
seed_notes[:1]
seed_indices = encode_seed(seed_notes, token2idx)
print(seed_indices) 

Notas no encontradas: 18
[520, 520, 5600, 11635, 203, 520, 641, 5600, 11635, 203, 520, 5228, 5600, 9689, 1534, 6572, 6986, 14561, 5228, 520, 14802, 14561, 14279, 520, 1855, 5012, 9067, 14561, 14279, 1855, 9689, 5012, 6986, 12292, 11635, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 7349, 520, 11635, 5600, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 1855, 5012, 5600, 1855, 14879, 12174, 520, 520, 5600, 11635, 203, 520, 641, 5600, 11635, 203, 520, 5228, 5600, 9689, 1534, 6572, 6986, 14561, 5228, 520, 14802, 14561, 14279, 520, 1855, 5012, 9067, 14561, 14279, 1855, 9689, 5012, 6986, 12292, 11635, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 7349, 520, 11635, 5600, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 1855, 5012, 5600, 1855, 14879, 12174]


In [7]:
gen_indices = generate_music_auto(model, seed_indices, gen_len=100, temperature=0.7)
idx2token = {v: k for k, v in token2idx.items()}
gen_tokens = sequence_to_tokens(gen_indices, idx2token)

print(gen_indices)

[520, 520, 5600, 11635, 203, 520, 641, 5600, 11635, 203, 520, 5228, 5600, 9689, 1534, 6572, 6986, 14561, 5228, 520, 14802, 14561, 14279, 520, 1855, 5012, 9067, 14561, 14279, 1855, 9689, 5012, 6986, 12292, 11635, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 7349, 520, 11635, 5600, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 1855, 5012, 5600, 1855, 14879, 12174, 520, 520, 5600, 11635, 203, 520, 641, 5600, 11635, 203, 520, 5228, 5600, 9689, 1534, 6572, 6986, 14561, 5228, 520, 14802, 14561, 14279, 520, 1855, 5012, 9067, 14561, 14279, 1855, 9689, 5012, 6986, 12292, 11635, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 7349, 520, 11635, 5600, 203, 11933, 5228, 11933, 12292, 11635, 1534, 6986, 6572, 1855, 5600, 11635, 203, 520, 5228, 11933, 1855, 5012, 5600, 1855, 14879, 12174, np.int64(6986), np.int64(12292), np.int64(12292), np.int6

**Cambiar nombre del archivo para diferenciarlos**

In [10]:
# Convertir a music21 stream y guardar MIDI
gen_stream = build_stream_from_sequence_tokens(gen_tokens)
midi_fp = "generated/generated_lullaby.mid"
gen_stream.write('midi', fp=midi_fp)
print("Generado MIDI:", midi_fp)

Generado MIDI: generated/generated_lullaby.mid


**Muestra de notas no encontradas**

Esto para verificar que notas no se encuentran y buscar soluciones como las descritas en mi mensaje

> No tiene nada que ver con lo ya generado, es código que usé para depurar y ver problemas antes de generar una canción

In [9]:
seed_encoded = []
for n in seed_notes:
    n_clean = n.strip() 
    if n_clean in token2idx:
        seed_encoded.append(token2idx[n_clean])
        # print(f"Nota {n_clean} encontrada y añadida.")
    else:
        seed_encoded.append(token2idx["<UNK>"])
        # print(f"Nota limpia '{n_clean}' no encontrada (original: '{n}').")

print(seed_encoded[:10])

[520, 520, 5600, 11635, 203, 520, 641, 5600, 11635, 203]
